In [1]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollama_model_client = OllamaChatCompletionClient(
    model="llama3.2"
)

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('GOOGLE_API_KEY')

In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

gemini_model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key 
)

In [4]:
from autogen_agentchat.agents import AssistantAgent

agentX = AssistantAgent(
    "primary",
    model_client=ollama_model_client,
    system_message="You are a helpful AI assistant.",
)

agentY = AssistantAgent(
    "critic" , 
    model_client=gemini_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

## Creating a Team

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

text_termination = TextMentionTermination("APPROVE")
termination_condition = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)

team = RoundRobinGroupChat(
    [agentX , agentY], 
     # max_turns=15 # -----># maximum number of Message before it stops between the agents.
    termination_condition=text_termination
)

## Running a Team

In [6]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)
print(result.messages)

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 10, 36, 553623, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=100), metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 10, 41, 945885, tzinfo=datetime.timezone.utc), content="As summer's warmth begins to fade,\nGolden leaves in rustling shade,\nCrisp air whispers through the trees,\nA symphony of nature's breeze.\n\nAmber hues and crimson bright,\nDance upon the autumn light,\nThe scent of woodsmoke fills the air,\nAs earthy tones are shared with care.\n\nThe season's fleeting, yet so grand,\nA moment's beauty, in this land,\nFalling leaves, a colorful sight,\nA final burst of summer's delight.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=133, completion_tokens=3), metadat

In [7]:
await team.reset()  # Reset the team for a new task.
# This method will clear the team’s state, including all agents. 
# It will call the each agent’s on_reset() method to clear the agent’s state.

async for chunk in team.run_stream(task="Write a short poem about the fall season."):
    print(chunk)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 16, 5, 10, 44, 141129, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=109) metadata={} created_at=datetime.datetime(2025, 7, 16, 5, 10, 48, 919593, tzinfo=datetime.timezone.utc) content="As autumn leaves begin to sway,\nGolden hues and crimson play,\nThe air is crisp, the wind is cold,\nNature's final dance before winter's hold.\n\nThe trees stand tall, their branches high,\nA fiery crown against the gray sky,\nTheir leaves rustle down to earth below,\nA crunchy carpet for the winds to blow.\n\nThe scent of woodsmoke wafts through air,\nAs earthy smells and spices fill with care,\nA season of decay, yet still so bright,\nA time for change, a time for delight." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=145, completion_tokens=3) metadata={} cre

## Observing a Team

In [8]:
from autogen_agentchat.base import TaskResult

# When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

await team.reset()
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 16, 5, 10, 49, 607553, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=99) metadata={} created_at=datetime.datetime(2025, 7, 16, 5, 10, 53, 929089, tzinfo=datetime.timezone.utc) content="As autumn leaves begin to fade,\nGolden hues and crimson shades,\nThe trees stand tall, their limbs outstretched,\nA final dance before winter's sketch.\n\nThe air is crisp, the wind is cold,\nNature's palette, young and old,\nThe scent of woodsmoke fills the air,\nAs earthy smells and memories share.\n\nThe sun sets early now each day,\nA fiery glow in fading ray,\nThe stars appear with gentle light,\nA magical night, a peaceful sight." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=133, completion_tokens=3) metadata={} created_at=datetime.datetime(2025, 7, 16, 5,

In [9]:
import asyncio

async def work(x):
    await asyncio.sleep(x)
    return f"Finished after {x} seconds"

# Run two tasks concurrently
task1 = asyncio.create_task(work(2))
task2 = asyncio.create_task(work(3))

# Wait for both
result1 = await task1
result2 = await task2


In [10]:
import time

async def work(x):
    await asyncio.sleep(x)
    return f"Finished after {x} seconds"

def func():
    time.sleep(2)  # BLOCKS the entire event loop (sync code in async context)

task1 = asyncio.create_task(work(2))    # Starts now (0s)
task2 = asyncio.create_task(work(3))    # Starts now (0s)

await task1                             # Waits 2s, during which task2 keeps running
func()                                  # BLOCKS for 2s (from t=2s to t=4s)
task3 = asyncio.create_task(work(2))    # Starts at t=4s
await task2                             # task2 was supposed to finish at 3s, but delayed to 4s
task4 = asyncio.create_task(work(3))    # Starts at ~t=4s
await task3                             # task3 finishes at ~6s
func()                                  # BLOCKS again for 2s (from t=6s to t=8s)
await task4                             # task4 finishes at ~9s (delayed by func())


'Finished after 3 seconds'

In [ ]:
import asyncio

async def work(x):
    await asyncio.sleep(x)
    return f"Finished after {x} seconds"

task1 = asyncio.create_task(work(3))   # starts now, runs for 3s
task2 = asyncio.create_task(work(2))   # starts now, runs for 2s

await task1                           # wait for task1 (3s)
task3 = asyncio.create_task(work(2)) # starts after task1 completes (~3s)
await task2                           # wait for task2 (2s)
task4 = asyncio.create_task(work(3)) # starts after task2 completes (~3s or earlier)
await task3                          # wait for task3 (2s)
await task4                          # wait for task4 (3s)


# | Time | What happens                                                                        |
# | ---- | ----------------------------------------------------------------------------------- |
# | 0s   | `task1` and `task2` start simultaneously. `task1` runs for 3s, `task2` runs for 2s. |
# | 0–2s | Both tasks running concurrently.                                                    |
# | 2s   | `task2` finishes.                                                                   |
# | 0–3s | `await task1` is waiting for `task1` to finish. Your coroutine is paused here.      |
# | 3s   | `task1` finishes. Your coroutine resumes after `await task1`.                       |
# | 3s   | `task3` starts (2s).                                                                |
# | 3s   | `await task2` — but `task2` already finished at 2s, so this returns immediately.    |
# | 3s   | `task4` starts (3s).                                                                |
# | 3s   | Coroutine hits `await task3` and waits for it to finish (2s).                       |
# | 3–5s | `task3` and `task4` run concurrently.                                               |
# | 5s   | `task3` finishes, coroutine resumes and hits `await task4`.                         |
# | 5–6s | Coroutine waits for `task4` to finish (1s left).                                    |
# | 6s   | `task4` finishes, coroutine ends.                                                   |

# Important notes:
# await task1 pauses your coroutine until task1 finishes (3s). Meanwhile, task2 is running concurrently.
# When you await task2 after task1, task2 is already done, so it returns immediately.
# You start task3 after task1 finishes, and start task4 after task2 finishes (which is earlier).
# task3 and task4 run concurrently between 3s and 5s.
# Your coroutine awaits task3, then awaits task4.
# The total run time is about 6 seconds.

# | Task  | Start time | Duration | End time | Notes                            |
# | ----- | ---------- | -------- | -------- | -------------------------------- |
# | task1 | 0s         | 3s       | 3s       | First awaited, blocks until done |
# | task2 | 0s         | 2s       | 2s       | Runs concurrently with task1     |
# | task3 | 3s         | 2s       | 5s       | Starts after task1 finishes      |
# | task4 | 3s         | 3s       | 6s       | Starts after task2 finishes      |

# Visual analogy:
# You start two timers: one 3-second, one 2-second.
# You wait fully for the 3-second timer (await task1), while the 2-second timer runs in parallel.
# After the 3-second timer, you start another 2-second timer.
# You check if the 2-second timer (task2) is done — yes, so no wait.
# Start the last 3-second timer.
# Wait sequentially for the last two timers to finish, which overlap partially.


In [11]:
from autogen_agentchat.ui import Console

await team.reset() 
await Console(team.run_stream(task="Write a short poem about the fall season.")) 

# team.run(...) is an awaitable coroutine.  ==> Blocking
# team.run_stream(...) returns an async generator.e.

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
As summer's warmth begins to fade,
The trees don cloaks of golden shade,
Their leaves, like nature's confetti bright,
Drifting down in whispers of the night.

The air is crisp, the wind is cold,
A hint of woodsmoke, young and old,
The scent of earth and leaves so fine,
As autumn's palette paints its divine.

The sun sets early, a fiery glow,
A final dance before the winter's snow,
The stars appear, like diamonds bright,
In the fall's fleeting, golden light.
---------- TextMessage (critic) ----------
APPROVE



TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 5, 699974, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=110), metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 10, 636035, tzinfo=datetime.timezone.utc), content="As summer's warmth begins to fade,\nThe trees don cloaks of golden shade,\nTheir leaves, like nature's confetti bright,\nDrifting down in whispers of the night.\n\nThe air is crisp, the wind is cold,\nA hint of woodsmoke, young and old,\nThe scent of earth and leaves so fine,\nAs autumn's palette paints its divine.\n\nThe sun sets early, a fiery glow,\nA final dance before the winter's snow,\nThe stars appear, like diamonds bright,\nIn the fall's fleeting, golden light.", type='TextMessage'), TextMessage(source='critic', models_usage=Req

In [12]:
# Example analogy:
# - You order a coffee and wait (await), but the barista keeps making coffee for others
#  meanwhile (event loop runs other tasks).
# - You stop working until your coffee arrives (your coroutine paused).
# - Other customers get served without waiting for you (other coroutines continue).


# | Blocking type  | Effect                                           |
# | -------------- | ------------------------------------------------ |
# | `await`        | Pauses current coroutine, event loop runs others |
# | `time.sleep()` | Blocks entire thread, nothing else runs          |


## Stopping the Team 

In [13]:
from autogen_agentchat.conditions import ExternalTermination
import asyncio

external_termination = ExternalTermination()

team = RoundRobinGroupChat(
    [agentX, agentY],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- TextMessage (user) ----------
Write a short poem about the fall season.


---------- TextMessage (primary) ----------
Amber hues upon the trees
As nature's final dance begins to freeze
Golden leaves that rustle free
A symphony of autumn glee

Crisp air whispers through the night
Woodsmoke wafts, a warm delight
The scent of earth and leaves so bright
Invigorates the senses in fall's light


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 12, 648574, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=169, completion_tokens=69), metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 15, 869371, tzinfo=datetime.timezone.utc), content="Amber hues upon the trees\nAs nature's final dance begins to freeze\nGolden leaves that rustle free\nA symphony of autumn glee\n\nCrisp air whispers through the night\nWoodsmoke wafts, a warm delight\nThe scent of earth and leaves so bright\nInvigorates the senses in fall's light", type='TextMessage')], stop_reason='External termination requested')

In [14]:
# asyncio.create_task(Console(team.run_stream(...)))

# 1. "Run the stream"
# 2. "Pipe the output into Console()"
# 3. "Do it without waiting immediately — I’ll await it when I’m ready"
# 4 .Then [await run] lets you pause and wait for it to complete.

In [15]:
await team.reset()

task = asyncio.create_task(team.run(task="Write a short poem about the fall season."))
result = await task

print(result)
print(result.messages)


messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 15, 895491, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=110), metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 20, 693805, tzinfo=datetime.timezone.utc), content="As summer's warmth begins to fade,\nThe trees don crimson, amber shade.\nGolden leaves rustle, soft and low,\nA gentle whisper as they let go.\n\nThe air is crisp, the winds do blow,\nAnd nature's final dance begins to show.\nThe scent of woodsmoke wafts through the night,\nAs fall's last breath takes flight.\n\nThe season's palette, vibrant and bright,\nPaints a picture of cozy delight.\nA time for harvest, for warmth, for rest,\nFall's sweet magic is at its best.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(promp

In [16]:
# Use await directly:
# When you want the result right away
# No other tasks need to run in parallel

await team.reset()
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Waits till done before moving forward

# Use asyncio.create_task():
# When you want to start a task in the background
# Do other things before waiting
# Handle multiple tasks concurrently

await team.reset()
task = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))
# do something else...
await task  # Now wait for it



---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
Golden leaves dance in the breeze,
Crimson and amber, nature's ease.
The air is crisp, the wind is cold,
As summer's warmth begins to unfold.

The scent of woodsmoke fills the air,
As trees prepare for their winter lair.
The earthy smell of fallen ground,
Invigorates the senses all around.

The sun sets early, softly low,
Painting the sky with hues of gold and red to show.
A season of change, a time to gaze,
On nature's beauty in its autumn daze.
---------- TextMessage (critic) ----------
APPROVE

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
As summer's warmth begins to fade,
The trees don golden hues displayed,
Crimson, amber, and honey bright,
A colorful finale to the light.

The air is crisp, the winds do blow,
And nature's palette starts to show,
Leaves rustle, twirling to the

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 26, 797798, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=107), metadata={}, created_at=datetime.datetime(2025, 7, 16, 5, 11, 31, 393641, tzinfo=datetime.timezone.utc), content="As summer's warmth begins to fade,\nThe trees don golden hues displayed,\nCrimson, amber, and honey bright,\nA colorful finale to the light.\n\nThe air is crisp, the winds do blow,\nAnd nature's palette starts to show,\nLeaves rustle, twirling to the ground,\nAs earthy scents and woods resound.\n\nThe sun sets early, casting low,\nA warm orange glow on all below,\nThe season's change, a time of rest,\nA fall farewell, before winter's nest.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=142, c

## Resuming in a Team

In [ ]:
await Console(team.run_stream())# Resume the team to continue the last task.


---------- TextMessage (primary) ----------
I'm glad you're happy with the poem. If you need anything else or have any other requests, feel free to ask!
---------- TextMessage (critic) ----------
Okay, I'm ready for your next request.  Let me know what you'd like.

---------- TextMessage (primary) ----------
I'd love to have a conversation with you about books or authors. Would you like some book recommendations, discuss a specific author or genre, or perhaps explore a particular theme in literature? Or maybe you just want to chat about reading and writing in general?

Let me know, and I'll do my best to assist you!
---------- TextMessage (critic) ----------
I'd love to chat about books!  I'm particularly interested in exploring a specific author or genre.  What author or genre are you interested in discussing?  Perhaps we could even talk about how certain themes are explored in different works.  Let me know!

---------- TextMessage (primary) ----------
That sounds like a fantastic con

## Aborting a Team

In [40]:
from autogen_core import CancellationToken
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
await team.reset()
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

cancellation_token.cancel()
# It is used to programmatically cancel the ongoing team.run(...) task.

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

# Different from stopping a team, aborting a team will immediately stop the team 
# and raise a CancelledError exception.


# | Stopping the team (soft stop)       | Aborting the team (immediate cancel)                      |
# | ----------------------------------- | --------------------------------------------------------- |
# | Stops after finishing current steps | Stops **immediately** by raising `CancelledError`         |
# | Allows cleanup or finishing work    | Forces immediate exit, no cleanup                         |
# | No exception raised                 | Raises `asyncio.CancelledError` to indicate forced cancel |

# CancellationToken lets you programmatically cancel a running team.run() task.
# Cancelling triggers CancelledError exception in the running task.
# You must catch CancelledError to handle the cancellation gracefully.
# This pattern helps you abort long-running tasks safely and responsively.


Task was cancelled.


In [48]:
cancellation_token2 = CancellationToken()

await team.reset()

run = asyncio.create_task(
    team.run(
        task="Hey how are you",
        cancellation_token=cancellation_token2,
    )
)

result = await run
print(result)

# if you use the same cancellation token then it will not work and give error.

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 9, 3, 4, 227296, tzinfo=datetime.timezone.utc), content='Hey how are you', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=65), metadata={}, created_at=datetime.datetime(2025, 7, 5, 9, 3, 7, 126687, tzinfo=datetime.timezone.utc), content="I'm doing well, thank you for asking! I'm a large language model, so I don't have emotions or feelings in the way that humans do, but I'm always happy to chat and help with any questions or topics you'd like to discuss. How about you? How's your day going so far?", type='TextMessage')] stop_reason='The group chat is stopped.'


In [53]:
import asyncio
from autogen_core import CancellationToken

async def run_and_cancel_after_delay():

    cancellation_token = CancellationToken()

    await team.reset()

    # Start running the team with run_stream
    run_stream = team.run_stream(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )

    # Create an async task to iterate messages from run_stream
    async def consume_stream():
        async for message in run_stream:
            print("Message:", message)

    consumer_task = asyncio.create_task(consume_stream())

    # Wait for 3 seconds before cancelling
    await asyncio.sleep(3)

    print("Cancelling the task now...")
    cancellation_token.cancel()

    try:
        await consumer_task
    except asyncio.CancelledError:
        print("Task was cancelled.")

# Run the async function
await run_and_cancel_after_delay()



# | Time | Activity                                                  |
# | ---- | --------------------------------------------------------- |
# | 0s   | `team.run_stream` starts streaming output                 |
# | 0-3s | `consume_stream` prints chunks as they arrive             |
# | 3s   | Cancellation requested with `cancellation_token.cancel()` |
# | 3s+  | Team stops, `consume_stream` raises CancelledError        |
# | End  | Cancellation handled, "Task was cancelled." printed       |

Message: source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 5, 9, 6, 17, 854687, tzinfo=datetime.timezone.utc) content='Translate the poem to Spanish.' type='TextMessage'
Message: source='primary' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=42) metadata={} created_at=datetime.datetime(2025, 7, 5, 9, 6, 19, 793837, tzinfo=datetime.timezone.utc) content="I'd be happy to help, but I don't see a poem provided. Please share the poem you'd like me to translate, and I'll do my best to translate it into Spanish for you." type='TextMessage'
Message: messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 9, 6, 17, 854687, tzinfo=datetime.timezone.utc), content='Translate the poem to Spanish.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=42), metadata={}, created_at=datetime.datetime(2025, 7, 5, 9, 6, 19, 793837, tzinfo=dateti

# Single Agent in RoundRobinTeam